In [1]:
import numpy                    as np
import scipy.io.wavfile         as wav
import matplotlib.pyplot        as plt
import sklearn.decomposition    as skd
import math
import numpy.linalg as la

In [3]:
# load data and set some helper variables
D = np.load("../Data/digits-labels.npz")["d"]
(numRawFeatures, numDesiredFeatures) = D.shape
numDesiredFeatures = 36

In [2]:
def pca(centeredData):
    return [U,S,Vt] = la.svd(D, full_matrices = False )

In [ ]:
def ica(zp):
    return 

In [ ]:
# 1. do pca using svd to get eigen vectors and eigen values
# 2. scale those eigen vectors with some version of eigen values. this becomes Wp
# 3. do ica on Wp to get Wi
# 4. W = Wp @ Wi
# 5. plot the pinv(W)